In [1]:
msg=b'}21s00000000' # make nonce start with 3 b'\x00'

In [2]:
from pwn import remote

In [3]:
re=remote("127.0.0.1", 2333)

[x] Opening connection to 127.0.0.1 on port 2333
[x] Opening connection to 127.0.0.1 on port 2333: Trying 127.0.0.1
[+] Opening connection to 127.0.0.1 on port 2333: Done


In [4]:
re.sendline(msg.hex())

In [5]:
re.recvuntil("MITM: ")
l=re.recvline()
data1=bytes.fromhex(l.decode())
ct1 = data1[:36]
tag1 = data1[36:36+16]
nonce = data1[36+16:]
re.sendline((ct1+tag1+nonce[1:]).hex())

In [6]:
re.recvuntil("MITM: ")
l=re.recvline()
data2=bytes.fromhex(l.decode())
ct2 = data2[:12]
tag2 = data2[12:12+16]
re.sendline((ct2+tag2+nonce[2:]).hex())

In [7]:
R.<x> = PolynomialRing(GF(2))

In [8]:
F.<z> = GF(2^128, modulus=x^128+x^7+x^2+x+1)

In [9]:
def fromint(x):
    l=ZZ(x).digits(2)
    l += [0]*(128-len(l))
    return F(l[::-1])
def toint(x):
    l = bin(x.integer_representation())[:1:-1]
    l += "0"*(128-len(l))
    return int(l,2)

In [10]:
def ghash(aad, txt, hashkey):
    len_aad = len(aad)
    len_txt = len(txt)

    if 0 == len_aad % 16:
        data = aad
    else:
        data = aad + b'\x00' * (16 - len_aad % 16)
    if 0 == len_txt % 16:
        data += txt
    else:
        data += txt + b'\x00' * (16 - len_txt % 16)

    tag = 0
    assert len(data) % 16 == 0
    for i in range(len(data) // 16):
        tag += fromint(bytes_to_long(data[i * 16: (i + 1) * 16]))
        tag *= hashkey
    tag += fromint(((8 * len_aad) << 64) | (8 * len_txt))
    tag *= hashkey

    return tag

In [11]:
from Crypto.Cipher import AES
from Crypto.Util.number import bytes_to_long, long_to_bytes

In [12]:
P.<X>=PolynomialRing(F)

In [13]:
p=ghash(b"from client", ct1, X) - fromint(bytes_to_long(tag1)) + ghash(b"from server", ct2, X) - fromint(bytes_to_long(tag2))

In [14]:
r=choice(p.roots())[0]

In [15]:
re.recvuntil("MITM: ")
l=re.recvline()
data3=bytes.fromhex(l.decode())
ct3 = data3[:36]
tag3 = data3[36:36+16]

In [16]:
mct3 = bytearray(ct3)
for i in range(4):
    mct3[i+11] = ord("flag"[i]) ^^ ord("test"[i]) ^^ mct3[i+11]
mct3 = bytes(mct3)

In [17]:
mtag3 = long_to_bytes(toint(ghash(b"from client", ct3, r) + ghash(b"from client", mct3, r) + fromint(bytes_to_long(tag3))))

In [18]:
re.sendline((mct3+mtag3+nonce[3:]).hex())

In [19]:
re.recvuntil("MITM: ")

b'MITM: '

In [20]:
l=re.recvline()
print(l)

b'b613aab3c48c2c55713d14a388524e37e932f5577b62a9c6b80fb4c4fce853bcc3181e152c9e793c4c39ab744fc2\n'


In [21]:
data4=bytes.fromhex(l.decode())

In [22]:
for i in range(24):
    print(chr(data1[i] ^^ data4[i] ^^ b"ctf.server/test?message="[i]), end="")

flag{an_example_flag}úë	